In [1]:
import pandas as pd
import requests
import re
import datetime
from datetime import timedelta
import time
import numpy as np
from scipy.stats import norm
import math
from matplotlib import pyplot as plt
from datetime import timedelta
import statsmodels.formula.api as smf 

In [2]:
# read in df
data_df = pd.read_csv('city_df_2.csv')
data_df

,Unnamed: 0,Unnamed: 0.1,property_id,sale_datetime,property_type,sale_price,building_num_units,building_year_built,City,County,...,Geo ZipCode,Geo City,Returns,California AGI,Total Tax Liability,City_Geo,City_Name,Unemployment Rate,Labor Force,Employment
0,0,1,7569023033,2011-07-15 00:00:00,SINGLE FAMILY RESIDENCE,3000000,1.0,2009.0,ROLLING HILLS,Los Angeles,...,POINT (-118.4122917 33.7761192),POINT (-118.38398 33.80178),"12,833","3,141,898,382","241,422,440","-118.38398,33.80178",PALOS VERDES ESTATES,6.0,"5,200","4,900"
1,1,90,7548017014,2011-07-29 00:00:00,SINGLE FAMILY RESIDENCE,400004,1.0,2016.0,ROLLING HILLS ESTATES,Los Angeles,...,POINT (-118.4122917 33.7761192),POINT (-118.38398 33.80178),"12,833","3,141,898,382","241,422,440","-118.38398,33.80178",PALOS VERDES ESTATES,6.0,"5,200","4,900"
2,2,998,7545017014,2011-07-15 00:00:00,SINGLE FAMILY RESIDENCE,1414014,1.0,1961.0,PALOS VERDES EST,Los Angeles,...,POINT (-118.4122917 33.7761192),POINT (-118.38398 33.80178),"12,833","3,141,898,382","241,422,440","-118.38398,33.80178",PALOS VERDES ESTATES,6.0,"5,200","4,900"
3,3,1122,7551012021,2011-07-20 00:00:00,SINGLE FAMILY RESIDENCE,1550000,1.0,1978.0,ROLLING HILLS ESTATES,Los Angeles,...,POINT (-118.4122917 33.7761192),POINT (-118.38398 33.80178),"12,833","3,141,898,382","241,422,440","-118.38398,33.80178",PALOS VERDES ESTATES,6.0,"5,200","4,900"
4,4,1211,7545019016,2011-07-18 00:00:00,SINGLE FAMILY RESIDENCE,2072500,1.0,1964.0,PALOS VERDES EST,Los Angeles,...,POINT (-118.4122917 33.7761192),POINT (-118.38398 33.80178),"12,833","3,141,898,382","241,422,440","-118.38398,33.80178",PALOS VERDES ESTATES,6.0,"5,200","4,900"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013884,1013884,4388966,7044012034,2010-06-01 00:00:00,SINGLE FAMILY RESIDENCE,575000,1.0,1970.0,CERRITOS,Los Angeles,...,POINT (-118.043405 33.84783),POINT (-118.03727 33.84847),"7,069","438,026,232","16,488,433","-118.03727,33.84847",LA PALMA,6.8,"7,000","6,500"
1013885,1013885,4388969,7044012017,2015-05-12 00:00:00,SINGLE FAMILY RESIDENCE,725007,1.0,1970.0,CERRITOS,Los Angeles,...,POINT (-118.043405 33.84783),POINT (-118.03727 33.84847),"7,420","506,706,416","18,994,011","-118.03727,33.84847",LA PALMA,3.0,"7,300","7,100"
1013886,1013886,4388976,7044013014,2010-05-04 00:00:00,SINGLE FAMILY RESIDENCE,525005,1.0,1970.0,CERRITOS,Los Angeles,...,POINT (-118.043405 33.84783),POINT (-118.03727 33.84847),"7,069","438,026,232","16,488,433","-118.03727,33.84847",LA PALMA,7.0,"7,000","6,500"
1013887,1013887,4392443,3204085053,2021-05-12 00:00:00,SINGLE FAMILY RESIDENCE,535005,1.0,2007.0,LANCASTER,Los Angeles,...,POINT (-117.6694246 35.6279795),POINT (-117.669872 35.624808),511,"34,617,856","1,368,457","-117.669872,35.624808",RIDGECREST,4.0,"13,100","12,600"


In [3]:
data_df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'property_id', 'sale_datetime',
       'property_type', 'sale_price', 'building_num_units',
       'building_year_built', 'City', 'County', 'Address', 'zip_code',
       'Address_Google', 'closest_zip_1', 'distance_1', 'closest_zip_2',
       'distance_2', 'closest_zip_3', 'distance_3', 'Month', 'Year',
       'Geo ZipCode', 'Geo City', 'Returns', 'California AGI',
       'Total Tax Liability', 'City_Geo', 'City_Name', 'Unemployment Rate',
       'Labor Force', 'Employment'],
      dtype='object')

In [4]:
data_df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'building_num_units','Address'], inplace=True)

In [5]:
data_df.drop(columns=['Geo ZipCode', 'Geo City'], inplace=True)

In [6]:
data_df['Date'] = pd.to_datetime(data_df['sale_datetime'])

data_df.drop(columns=['sale_datetime'], inplace=True)

In [7]:
data_df.drop(columns=['County'])

,property_id,property_type,sale_price,building_year_built,City,zip_code,Address_Google,closest_zip_1,distance_1,closest_zip_2,...,Year,Returns,California AGI,Total Tax Liability,City_Geo,City_Name,Unemployment Rate,Labor Force,Employment,Date
0,7569023033,SINGLE FAMILY RESIDENCE,3000000,2009.0,ROLLING HILLS,90274,"0 CHESTNUT LN, ROLLING HILLS, CA 90274, USA",90275,1.37,90277,...,2011,"12,833","3,141,898,382","241,422,440","-118.38398,33.80178",PALOS VERDES ESTATES,6.0,"5,200","4,900",2011-07-15
1,7548017014,SINGLE FAMILY RESIDENCE,400004,2016.0,ROLLING HILLS ESTATES,90274,"1 MOCASSIN LN, ROLLING HILLS ESTATES, CA 90274...",90275,1.37,90277,...,2011,"12,833","3,141,898,382","241,422,440","-118.38398,33.80178",PALOS VERDES ESTATES,6.0,"5,200","4,900",2011-07-29
2,7545017014,SINGLE FAMILY RESIDENCE,1414014,1961.0,PALOS VERDES EST,90274,"1215 GRANVIA ALTAMIRA, PALOS VERDES EST, CA 90...",90275,1.37,90277,...,2011,"12,833","3,141,898,382","241,422,440","-118.38398,33.80178",PALOS VERDES ESTATES,6.0,"5,200","4,900",2011-07-15
3,7551012021,SINGLE FAMILY RESIDENCE,1550000,1978.0,ROLLING HILLS ESTATES,90274,"13 BRIDLEWOOD CIR, ROLLING HILLS ESTATES, CA 9...",90275,1.37,90277,...,2011,"12,833","3,141,898,382","241,422,440","-118.38398,33.80178",PALOS VERDES ESTATES,6.0,"5,200","4,900",2011-07-20
4,7545019016,SINGLE FAMILY RESIDENCE,2072500,1964.0,PALOS VERDES EST,90274,"1308 VIA MARGARITA, PALOS VERDES EST, CA 90274...",90275,1.37,90277,...,2011,"12,833","3,141,898,382","241,422,440","-118.38398,33.80178",PALOS VERDES ESTATES,6.0,"5,200","4,900",2011-07-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013884,7044012034,SINGLE FAMILY RESIDENCE,575000,1970.0,CERRITOS,90623,"7652 EL TORO CIR, CERRITOS, CA 90623, USA",90715,1.71,90630,...,2010,"7,069","438,026,232","16,488,433","-118.03727,33.84847",LA PALMA,6.8,"7,000","6,500",2010-06-01
1013885,7044012017,SINGLE FAMILY RESIDENCE,725007,1970.0,CERRITOS,90623,"7662 EL LOBO CIR, CERRITOS, CA 90623, USA",90715,1.71,90630,...,2015,"7,420","506,706,416","18,994,011","-118.03727,33.84847",LA PALMA,3.0,"7,300","7,100",2015-05-12
1013886,7044013014,SINGLE FAMILY RESIDENCE,525005,1970.0,CERRITOS,90623,"7682 EL RIO VERDE CIR, CERRITOS, CA 90623, USA",90715,1.71,90630,...,2010,"7,069","438,026,232","16,488,433","-118.03727,33.84847",LA PALMA,7.0,"7,000","6,500",2010-05-04
1013887,3204085053,SINGLE FAMILY RESIDENCE,535005,2007.0,LANCASTER,93556,"7111 W AVENUE L4, LANCASTER, CA 93556, USA",93535,69.51,93534,...,2021,511,"34,617,856","1,368,457","-117.669872,35.624808",RIDGECREST,4.0,"13,100","12,600",2021-05-12


In [8]:
data_df = pd.get_dummies(data_df, columns=['property_type'], drop_first=True, dtype=int)

In [9]:
data_df.head()

,property_id,sale_price,building_year_built,City,County,zip_code,Address_Google,closest_zip_1,distance_1,closest_zip_2,...,California AGI,Total Tax Liability,City_Geo,City_Name,Unemployment Rate,Labor Force,Employment,Date,property_type_SINGLE FAMILY DWELLING,property_type_SINGLE FAMILY RESIDENCE
0,7569023033,3000000,2009.0,ROLLING HILLS,Los Angeles,90274,"0 CHESTNUT LN, ROLLING HILLS, CA 90274, USA",90275,1.37,90277,...,"3,141,898,382","241,422,440","-118.38398,33.80178",PALOS VERDES ESTATES,6.0,"5,200","4,900",2011-07-15,0,1
1,7548017014,400004,2016.0,ROLLING HILLS ESTATES,Los Angeles,90274,"1 MOCASSIN LN, ROLLING HILLS ESTATES, CA 90274...",90275,1.37,90277,...,"3,141,898,382","241,422,440","-118.38398,33.80178",PALOS VERDES ESTATES,6.0,"5,200","4,900",2011-07-29,0,1
2,7545017014,1414014,1961.0,PALOS VERDES EST,Los Angeles,90274,"1215 GRANVIA ALTAMIRA, PALOS VERDES EST, CA 90...",90275,1.37,90277,...,"3,141,898,382","241,422,440","-118.38398,33.80178",PALOS VERDES ESTATES,6.0,"5,200","4,900",2011-07-15,0,1
3,7551012021,1550000,1978.0,ROLLING HILLS ESTATES,Los Angeles,90274,"13 BRIDLEWOOD CIR, ROLLING HILLS ESTATES, CA 9...",90275,1.37,90277,...,"3,141,898,382","241,422,440","-118.38398,33.80178",PALOS VERDES ESTATES,6.0,"5,200","4,900",2011-07-20,0,1
4,7545019016,2072500,1964.0,PALOS VERDES EST,Los Angeles,90274,"1308 VIA MARGARITA, PALOS VERDES EST, CA 90274...",90275,1.37,90277,...,"3,141,898,382","241,422,440","-118.38398,33.80178",PALOS VERDES ESTATES,6.0,"5,200","4,900",2011-07-18,0,1


In [10]:
data_df = data_df.loc[data_df['sale_price'] > 10]

In [11]:
data_df

,property_id,sale_price,building_year_built,City,County,zip_code,Address_Google,closest_zip_1,distance_1,closest_zip_2,...,California AGI,Total Tax Liability,City_Geo,City_Name,Unemployment Rate,Labor Force,Employment,Date,property_type_SINGLE FAMILY DWELLING,property_type_SINGLE FAMILY RESIDENCE
0,7569023033,3000000,2009.0,ROLLING HILLS,Los Angeles,90274,"0 CHESTNUT LN, ROLLING HILLS, CA 90274, USA",90275,1.37,90277,...,"3,141,898,382","241,422,440","-118.38398,33.80178",PALOS VERDES ESTATES,6.0,"5,200","4,900",2011-07-15,0,1
1,7548017014,400004,2016.0,ROLLING HILLS ESTATES,Los Angeles,90274,"1 MOCASSIN LN, ROLLING HILLS ESTATES, CA 90274...",90275,1.37,90277,...,"3,141,898,382","241,422,440","-118.38398,33.80178",PALOS VERDES ESTATES,6.0,"5,200","4,900",2011-07-29,0,1
2,7545017014,1414014,1961.0,PALOS VERDES EST,Los Angeles,90274,"1215 GRANVIA ALTAMIRA, PALOS VERDES EST, CA 90...",90275,1.37,90277,...,"3,141,898,382","241,422,440","-118.38398,33.80178",PALOS VERDES ESTATES,6.0,"5,200","4,900",2011-07-15,0,1
3,7551012021,1550000,1978.0,ROLLING HILLS ESTATES,Los Angeles,90274,"13 BRIDLEWOOD CIR, ROLLING HILLS ESTATES, CA 9...",90275,1.37,90277,...,"3,141,898,382","241,422,440","-118.38398,33.80178",PALOS VERDES ESTATES,6.0,"5,200","4,900",2011-07-20,0,1
4,7545019016,2072500,1964.0,PALOS VERDES EST,Los Angeles,90274,"1308 VIA MARGARITA, PALOS VERDES EST, CA 90274...",90275,1.37,90277,...,"3,141,898,382","241,422,440","-118.38398,33.80178",PALOS VERDES ESTATES,6.0,"5,200","4,900",2011-07-18,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013884,7044012034,575000,1970.0,CERRITOS,Los Angeles,90623,"7652 EL TORO CIR, CERRITOS, CA 90623, USA",90715,1.71,90630,...,"438,026,232","16,488,433","-118.03727,33.84847",LA PALMA,6.8,"7,000","6,500",2010-06-01,0,1
1013885,7044012017,725007,1970.0,CERRITOS,Los Angeles,90623,"7662 EL LOBO CIR, CERRITOS, CA 90623, USA",90715,1.71,90630,...,"506,706,416","18,994,011","-118.03727,33.84847",LA PALMA,3.0,"7,300","7,100",2015-05-12,0,1
1013886,7044013014,525005,1970.0,CERRITOS,Los Angeles,90623,"7682 EL RIO VERDE CIR, CERRITOS, CA 90623, USA",90715,1.71,90630,...,"438,026,232","16,488,433","-118.03727,33.84847",LA PALMA,7.0,"7,000","6,500",2010-05-04,0,1
1013887,3204085053,535005,2007.0,LANCASTER,Los Angeles,93556,"7111 W AVENUE L4, LANCASTER, CA 93556, USA",93535,69.51,93534,...,"34,617,856","1,368,457","-117.669872,35.624808",RIDGECREST,4.0,"13,100","12,600",2021-05-12,0,1


In [12]:
data_df.drop(columns=['City_Geo', 'County', 'City'], inplace=True)

In [13]:
data_df.drop(columns=['Employment', 'Total Tax Liability'], inplace=True)

data_df['Returns'] = data_df['Returns'].str.replace(',', '').astype(int)
data_df['California AGI'] = data_df['California AGI'].str.replace(',', '').astype(int)
data_df['Unemployment Rate'] = data_df['Unemployment Rate'].astype(float)
data_df['Labor Force'] = data_df['Labor Force'].str.replace(',', '').astype(int)

In [14]:
data_df.sort_values(by='Date', inplace=True)

In [16]:
data_df.loc[data_df['sale_price'] == 1000000]

,property_id,sale_price,building_year_built,zip_code,Address_Google,closest_zip_1,distance_1,closest_zip_2,distance_2,closest_zip_3,...,Returns,California AGI,City_Name,Unemployment Rate,Labor Force,Date,property_type_SINGLE FAMILY DWELLING,property_type_SINGLE FAMILY RESIDENCE,sale_price_Prev,change_sale_price
927241,4464012040,1000000,1995.0,90265,"31435 BIRDELLA RD, MALIBU, CA 90265, USA",91301,7.03,91302,7.76,90290,...,7780,1783772821,MALIBU,9.2,5000,2010-02-09,0,1,173000.0,827000.0
220299,7209029001,1000000,1949.0,90806,"450 W 20TH ST, LONG BEACH, CA 90806, USA",90755,1.26,90813,1.43,90807,...,15761,534884209,LONG BEACH,14.2,235800,2010-02-10,0,0,595000.0,405000.0
393165,4273020021,1000000,1949.0,90405,"1811 OCEAN PARK BLVD, SANTA MONICA, CA 90405, USA",90291,1.11,90404,1.34,90401,...,15080,1350204052,SANTA MONICA,11.2,53600,2010-02-19,0,0,300000.0,700000.0
297552,632710046,1000000,2000.0,92211,"792 MISSION CREEK DR, PALM DESERT, CA 92211, USA",92276,2.76,92260,3.87,92270,...,10841,531224556,PALM DESERT,10.5,21500,2010-02-23,1,0,1100000.0,-100000.0
307344,674680027,1000000,2004.0,92270,"5 LEDGESTONE LN, RANCHO MIRAGE, CA 92270, USA",92276,2.75,92260,4.30,92211,...,7906,574011474,RANCHO MIRAGE,19.7,5900,2010-02-26,1,0,86000.0,914000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
922676,955520007,1000000,2011.0,92592,"32571 ROTHENBERG DR, TEMECULA, CA 92592, USA",92591,5.08,92590,5.95,92563,...,34492,2955396832,TEMECULA,4.8,55000,2021-10-01,1,0,1320013.0,-320013.0
525897,271040067,1000000,2001.0,92504,"16041 HERITAGE GROVE RD, RIVERSIDE, CA 92504, USA",92506,1.65,92505,3.51,92509,...,25243,1425159814,RIVERSIDE,5.8,156200,2021-10-06,1,0,1075000.0,-75000.0
525920,227283014,1000000,1930.0,92504,"7172 EL PADRO ST, RIVERSIDE, CA 92504, USA",92506,1.65,92505,3.51,92509,...,25243,1425159814,RIVERSIDE,5.8,156200,2021-10-07,1,0,745007.0,254993.0
305892,620361055,1000000,1999.0,92260,"24 MERILL DR, PALM DESERT, CA 92260, USA",92210,2.30,92211,3.87,92270,...,14937,967905451,PALM DESERT,6.9,25300,2021-10-19,1,0,495000.0,505000.0


In [17]:
data_df = data_df.loc[data_df['sale_price'] > 50000]

In [18]:
data_df = data_df.loc[data_df['sale_price'] < 999999999]

In [19]:
data_df.sort_values(by='sale_price')

,property_id,sale_price,building_year_built,zip_code,Address_Google,closest_zip_1,distance_1,closest_zip_2,distance_2,closest_zip_3,...,Returns,California AGI,City_Name,Unemployment Rate,Labor Force,Date,property_type_SINGLE FAMILY DWELLING,property_type_SINGLE FAMILY RESIDENCE,sale_price_Prev,change_sale_price
856965,661051015,50400,2018.0,92240,"8751 CLUBHOUSE DR, DESERT HOT SPRINGS, CA 9224...",92241,3.25,92258,3.90,92262,...,13640,422557586,DESERT HOT SPRINGS,7.0,11300,2016-12-21,1,0,280000.0,-229600.0
562493,447280053,50500,1956.0,92544,"26963 YALE ST, HEMET, CA 92544, USA",92543,3.63,92583,5.37,92545,...,15126,660065795,HEMET,19.4,28400,2010-03-25,1,0,275002.0,-224502.0
562734,443181002,50500,1962.0,92543,"601 W DEVONSHIRE AVE, HEMET, CA 92543, USA",92545,2.23,92583,2.95,92544,...,9922,658614250,HEMET,18.3,28300,2011-10-28,1,0,215000.0,-164500.0
569929,552130069,50500,1963.0,92544,"26079 DOVERWOOD PL, HEMET, CA 92544, USA",92543,3.63,92583,5.37,92545,...,15153,665391668,HEMET,15.6,28000,2012-06-06,1,0,418004.0,-367504.0
645226,419280025,50500,1980.0,92220,"202 CLAIR CT, BANNING, CA 92220, USA",92230,5.25,92223,8.52,92320,...,11660,404823656,BANNING,12.5,10300,2010-07-01,1,0,600000.0,-549500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,7551039008,636376363,2019.0,90274,"6 PHILLIPS RANCH RD, ROLLING HILLS ESTATES, CA...",90275,1.37,90277,2.40,90505,...,12941,3410927610,PALOS VERDES ESTATES,3.2,5400,2015-07-02,0,1,636376363.0,0.0
119,7551037021,636376363,2019.0,90274,"7 MCCARRELL RANCH RD, PALOS VERDES ESTATES, CA...",90275,1.37,90277,2.40,90505,...,12941,3410927610,PALOS VERDES ESTATES,3.2,5400,2015-07-02,0,1,450000.0,635926363.0
50,7551039020,636376363,2018.0,90274,"15 PHILLIPS RANCH RD, ROLLING HILLS ESTATES, C...",90275,1.37,90277,2.40,90505,...,12941,3410927610,PALOS VERDES ESTATES,3.2,5400,2015-07-02,0,1,636376363.0,0.0
109,7551038031,636376363,2019.0,90274,"57 CHANDLER RANCH RD, ROLLING HILLS ESTATES, C...",90275,1.37,90277,2.40,90505,...,12941,3410927610,PALOS VERDES ESTATES,3.2,5400,2015-07-02,0,1,255002.0,636121361.0


In [20]:
data_df = data_df.loc[data_df['sale_price'] < 636376363]

In [21]:
data_df.sort_values(by='sale_price')

,property_id,sale_price,building_year_built,zip_code,Address_Google,closest_zip_1,distance_1,closest_zip_2,distance_2,closest_zip_3,...,Returns,California AGI,City_Name,Unemployment Rate,Labor Force,Date,property_type_SINGLE FAMILY DWELLING,property_type_SINGLE FAMILY RESIDENCE,sale_price_Prev,change_sale_price
856965,661051015,50400,2018.0,92240,"8751 CLUBHOUSE DR, DESERT HOT SPRINGS, CA 9224...",92241,3.25,92258,3.90,92262,...,13640,422557586,DESERT HOT SPRINGS,7.0,11300,2016-12-21,1,0,280000.0,-229600.0
647062,541043001,50500,1936.0,92220,"684 N HARGRAVE ST, BANNING, CA 92220, USA",92230,5.25,92223,8.52,92320,...,11753,412202026,BANNING,10.2,10200,2012-06-13,1,0,221002.0,-170502.0
652472,3148021007,50500,1981.0,93535,"43730 LENA CT, LANCASTER, CA 93535, USA",93534,2.22,93536,5.13,93550,...,24068,830902952,LANCASTER,13.2,62700,2012-02-02,0,1,250002.0,-199502.0
4363,6049020011,50500,1947.0,90002,"1235 E 99TH ST, LOS ANGELES, CA 90002, USA",90001,1.30,90052,1.58,90003,...,15919,380768039,LOS ANGELES,12.2,1980000,2011-12-29,0,0,160000.0,-109500.0
699243,3346027018,50500,1958.0,93591,"22317 E AVENUE O, PALMDALE, CA 93591, USA",93544,9.45,93543,9.97,93552,...,2084,25008988,PALMDALE,15.6,64200,2011-06-10,0,1,225002.0,-174502.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984215,691300012,395454545,2004.0,92203,"80455 CAMINO SAN LUCAS, INDIO, CA 92203, USA",92202,3.67,92201,4.18,92253,...,12479,808721728,INDIO,7.4,41000,2018-09-26,1,0,719007.0,394735538.0
260464,4211037053,472089175,2014.0,90094,"12760 W MILLENNIUM RD, LOS ANGELES, CA 90094, USA",90045,1.42,90066,1.47,90293,...,3523,508593709,LOS ANGELES,5.8,2002000,2016-02-10,0,0,7168571.0,464920604.0
260463,4211037054,472089175,2014.0,90094,"12722 MILLENNIUM DR, LOS ANGELES, CA 90094, USA",90045,1.42,90066,1.47,90293,...,3523,508593709,LOS ANGELES,5.8,2002000,2016-02-10,0,0,500005.0,471589170.0
872311,480841009,481775455,2018.0,92563,"30502 BOXLEAF LN, MURRIETA, CA 92563, USA",92591,2.97,92562,3.53,92590,...,27732,1800539293,MURRIETA,3.2,55600,2018-11-30,1,0,395000.0,481380455.0


In [22]:
data_df = data_df.loc[data_df['sale_price'] < 395454545]
data_df.sort_values(by='sale_price')

,property_id,sale_price,building_year_built,zip_code,Address_Google,closest_zip_1,distance_1,closest_zip_2,distance_2,closest_zip_3,...,Returns,California AGI,City_Name,Unemployment Rate,Labor Force,Date,property_type_SINGLE FAMILY DWELLING,property_type_SINGLE FAMILY RESIDENCE,sale_price_Prev,change_sale_price
856965,661051015,50400,2018.0,92240,"8751 CLUBHOUSE DR, DESERT HOT SPRINGS, CA 9224...",92241,3.25,92258,3.90,92262,...,13640,422557586,DESERT HOT SPRINGS,7.0,11300,2016-12-21,1,0,280000.0,-229600.0
576173,446161008,50500,1957.0,92543,"619 E MAYBERRY AVE, HEMET, CA 92543, USA",92545,2.23,92583,2.95,92544,...,9922,658614250,HEMET,17.4,28000,2011-05-05,1,0,140000.0,-89500.0
696661,3008032021,50500,1953.0,93550,"38587 9TH ST E, PALMDALE, CA 93550, USA",93552,4.48,93551,4.68,93535,...,25029,816784228,PALMDALE,15.6,63700,2012-02-27,0,1,510005.0,-459505.0
707649,3053012038,50500,1984.0,93550,"36433 HAROLD 1ST ST, PALMDALE, CA 93550, USA",93552,4.48,93551,4.68,93535,...,24041,5080982,PALMDALE,16.8,64700,2011-09-07,0,1,183000.0,-132500.0
571113,449161033,50500,1967.0,92544,"41046 MCDOWELL ST, HEMET, CA 92544, USA",92543,3.63,92583,5.37,92545,...,15153,665391668,HEMET,16.9,28300,2012-03-29,1,0,450004.0,-399504.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718701,8625016044,271555424,2016.0,91741,"271 CLEMENTINE CT, GLENDORA, CA 91741, USA",91740,1.38,91702,3.17,91773,...,12046,1051450237,GLENDORA,7.4,25000,2014-04-02,0,1,68956189.0,202599235.0
718700,8625017108,271555424,2016.0,91741,"268 SNAPDRAGON LN, GLENDORA, CA 91741, USA",91740,1.38,91702,3.17,91773,...,12046,1051450237,GLENDORA,7.4,25000,2014-04-02,0,1,430000.0,271125424.0
718714,8625017068,271555424,2016.0,91741,"332 SNAPDRAGON LN, GLENDORA, CA 91741, USA",91740,1.38,91702,3.17,91773,...,12046,1051450237,GLENDORA,7.4,25000,2014-04-02,0,1,248500.0,271306924.0
718659,8625017060,271555424,2016.0,91741,"1033 ELDERBERRY DR, GLENDORA, CA 91741, USA",91740,1.38,91702,3.17,91773,...,12046,1051450237,GLENDORA,7.4,25000,2014-04-02,0,1,539005.0,271016419.0


In [23]:
data_df = data_df.loc[data_df['sale_price'] < 100000000]

In [24]:
data_df = data_df.loc[data_df['sale_price'] < 10000000]

In [25]:
data_df.sort_values(by='sale_price')

,property_id,sale_price,building_year_built,zip_code,Address_Google,closest_zip_1,distance_1,closest_zip_2,distance_2,closest_zip_3,...,Returns,California AGI,City_Name,Unemployment Rate,Labor Force,Date,property_type_SINGLE FAMILY DWELLING,property_type_SINGLE FAMILY RESIDENCE,sale_price_Prev,change_sale_price
856965,661051015,50400,2018.0,92240,"8751 CLUBHOUSE DR, DESERT HOT SPRINGS, CA 9224...",92241,3.25,92258,3.90,92262,...,13640,422557586,DESERT HOT SPRINGS,7.0,11300,2016-12-21,1,0,280000.0,-229600.0
1010088,774163005,50500,1986.0,92253,"53830 AVENIDA VALLEJO, LA QUINTA, CA 92253, USA",92201,3.42,92210,4.87,92236,...,14944,997767669,LA QUINTA,5.1,17200,2014-10-20,1,0,460000.0,-409500.0
372391,349030080,50500,1952.0,92570,"27520 HAMMACK AVE, PERRIS, CA 92570, USA",92571,4.11,92587,5.37,92586,...,16476,593818495,PERRIS,18.7,28500,2011-11-29,1,0,170001.0,-119501.0
647062,541043001,50500,1936.0,92220,"684 N HARGRAVE ST, BANNING, CA 92220, USA",92230,5.25,92223,8.52,92320,...,11753,412202026,BANNING,10.2,10200,2012-06-13,1,0,221002.0,-170502.0
666072,3069011009,50500,2007.0,93535,"40636 151ST ST E, LAKE LOS ANGELES, CA 93535, USA",93534,2.22,93536,5.13,93550,...,22869,432236315,LANCASTER,13.9,63500,2011-07-27,0,1,218000.0,-167500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392674,4293018018,9975099,1922.0,90402,"704 PALISADES BEACH RD, SANTA MONICA, CA 90402...",90403,0.72,90401,1.42,90404,...,5401,3038139011,SANTA MONICA,4.3,56600,2019-04-05,0,1,748007.0,9227092.0
928004,4458004042,9980099,1949.0,90265,"23538 MALIBU COLONY RD, MALIBU, CA 90265, USA",91301,7.03,91302,7.76,90290,...,7788,1643714237,MALIBU,4.8,5200,2015-07-09,0,1,232002.0,9748097.0
335527,4391015012,9985099,1962.0,90210,"335 TROUSDALE PL, BEVERLY HILLS, CA 90210, USA",90069,1.68,90212,2.27,90077,...,10554,3858926372,BEVERLY HILLS,10.8,18000,2012-09-28,0,1,200002000.0,-190016901.0
336992,4341006003,9995099,1988.0,90210,"706 N ARDEN DR, BEVERLY HILLS, CA 90210, USA",90069,1.68,90212,2.27,90077,...,10537,4981933783,BEVERLY HILLS,6.9,18200,2015-08-28,0,1,507005.0,9488094.0


In [28]:
data_df['Zip_month'] = data_df.apply(lambda x: f"{x['Month']}-{x['Year']}", axis=1)

data_df['Zip_month'] = pd.to_datetime(data_df['Zip_month'])

In [29]:
zips = data_df['zip_code'].unique()

In [30]:
zip_code = zips[0]

In [31]:
lags = ['sale_price', 'building_year_built', 'Unemployment Rate', 'California AGI' ,'Returns', 'property_type_SINGLE FAMILY DWELLING', 'property_type_SINGLE FAMILY RESIDENCE']



In [34]:
final_df = pd.DataFrame()
for zip_code in zips:
    print(zip_code)
    
    # Get main zip data
    zip_df = data_df.loc[data_df['zip_code'] == zip_code].copy()
    zip_df.sort_values(by='Date', inplace=True)
    
    # Get closest zips
    closest_zip_1 = zip_df['closest_zip_1'].iloc[0]
    closest_zip_2 = zip_df['closest_zip_2'].iloc[0]
    closest_zip_3 = zip_df['closest_zip_3'].iloc[0]

    # Get data for closest zips
    closest_df = data_df.loc[data_df['zip_code'] == closest_zip_1]
    closest_df2 = data_df.loc[data_df['zip_code'] == closest_zip_2]
    closest_df3 = data_df.loc[data_df['zip_code'] == closest_zip_3]
    
    # Create monthly aggregates
    zip_dfs = {}
    zip_dfs[0] = zip_df.groupby('Zip_month').mean()
    zip_dfs[1] = closest_df.groupby('Zip_month').mean()
    zip_dfs[2] = closest_df2.groupby('Zip_month').mean()
    zip_dfs[3] = closest_df3.groupby('Zip_month').mean()
    
    # Add Zip_months and sort
    for df in zip_dfs.values():
        df['Zip_months'] = df.index
        df.sort_values(by='Zip_months', inplace=True)

    nearby_var = ['Unemployment Rate', 'sale_price', 'California AGI', 'Labor Force']

    # Process nearby zips (1, 2, 3)
    nearby_dict = {'Date': [], 'property_id': []}
    
    # Initialize all dictionary keys first
    for zip_num in [1, 2, 3]:
        for var in nearby_var:
            for lag in [1, 2, 3]:
                nearby_dict[f'zip_{zip_num}_{var}_{lag}'] = []

    # Main processing loop for nearby zips
    for _, z in zip_df.iterrows():
        zip_month = z['Zip_month']
        nearby_dict['Date'].append(z['Date'])
        nearby_dict['property_id'].append(z['property_id'])
        
        for zip_num in [1, 2, 3]:
            filtered_data = zip_dfs[zip_num].loc[zip_dfs[zip_num]['Zip_months'] < zip_month].tail(3)
            
            if len(filtered_data) < 3:
                for var in nearby_var:
                    for lag in [1, 2, 3]:
                        nearby_dict[f'zip_{zip_num}_{var}_{lag}'].append(np.nan)
            else:
                current_lag = 3
                for _, row_data in filtered_data.iterrows():
                    for var in nearby_var:
                        nearby_dict[f'zip_{zip_num}_{var}_{current_lag}'].append(row_data[var])
                    current_lag -= 1

    # Process own zip (zip_0)
    nearby_dict_own = {'Date': [], 'property_id': []}  # Initialize with both Date and property_id
    
    # Initialize keys for own zip
    for var in nearby_var:
        for lag in [1, 2, 3]:  # Note: only lags 1-3 for own zip
            nearby_dict_own[f'{var}_{lag}'] = []

    # Process own zip data
    for _, z in zip_df.iterrows():
        zip_month = z['Zip_month']
        filtered_data = zip_dfs[0].loc[zip_dfs[0]['Zip_months'] < zip_month].tail(3)
        
        if len(filtered_data) < 3:
            for var in nearby_var:
                for lag in [1, 2, 3]:
                    nearby_dict_own[f'{var}_{lag}'].append(np.nan)
        else:
            current_lag = 3
            for _, row_data in filtered_data.iterrows():
                for var in nearby_var:
                    nearby_dict_own[f'{var}_{current_lag}'].append(row_data[var])
                current_lag -= 1
        
        nearby_dict_own['Date'].append(z['Date'])
        nearby_dict_own['property_id'].append(z['property_id'])  # Add to nearby_dict_own, not nearby_dict
        
    # Convert to dataframes and merge
    zip_nearby = pd.DataFrame(nearby_dict)
    zip_lags = pd.DataFrame(nearby_dict_own)
    
    cols = zip_nearby.drop(columns=['Date', 'property_id']).columns

    for col in cols:

        alpha = 0.02

        min_periods = 5
        
        zip_nearby['ewm_mean'] = zip_nearby[col].shift(1).ewm(span=20, min_periods=min_periods).mean()
        zip_nearby['ewm_std'] = zip_nearby[col].shift(1).ewm(span=20, min_periods=min_periods).std()
        zip_nearby[col] = np.where(
            zip_nearby['ewm_std'] > 1e-8,
            (zip_nearby[col] - zip_nearby['ewm_mean']) / zip_nearby['ewm_std'],
            0 
        )
        
    zip_nearby.drop(columns=['ewm_mean', 'ewm_std'], inplace=True)
    
    cols = zip_lags.drop(columns=['Date', 'property_id']).columns

    for col in cols:

        alpha = 0.02
        
        min_periods = 5
        zip_lags['ewm_mean'] = zip_lags[col].shift(1).ewm(span=20, min_periods=min_periods).mean()
        zip_lags['ewm_std'] = zip_lags[col].shift(1).ewm(span=20, min_periods=min_periods).std()
        zip_lags[col] = np.where(
            zip_lags['ewm_std'] > 1e-8,
            (zip_lags[col] - zip_lags['ewm_mean']) / zip_lags['ewm_std'],
            0 
        )
        
    zip_lags.drop(columns=['ewm_mean', 'ewm_std'], inplace=True)

    
    zip_df = pd.merge(zip_df, zip_nearby, on=['Date', 'property_id'])
    zip_df = pd.merge(zip_df, zip_lags, on=['Date', 'property_id'])
    
    # Append to final dataframe
    final_df = pd.concat([final_df, zip_df], ignore_index=True)
    

91342
91767
93535
91601
93534
90032
90061
92592
91402
90031
92879
91335
90249
91746
91744
90247
91766
90248
91740
91606
91344
92210
92880
92881
91780
92503
91406
92555
90670
92545
90002
91711
92201
93551
92544
90650
90293
93591
93536
90049
90077
91340
92553
93550
90034
90047
90660
91770
92223
90278
92501
90255
90712
92504
91024
91790
90814
90810
90043
92506
91303
90008
90815
91214
91207
90804
91208
90813
91202
91367
91350
92240
91411
91405
92883
90018
90062
91773
90028
90056
91401
91307
91732
92557
90013
91364
90805
90803
90250
91321
90716
91789
91351
91302
90045
90059
91741
91765
91791
91387
90280
90019
91724
91010
90262
91722
90023
91706
90016
90807
92860
90746
91306
90003
92551
92562
90638
91331
90026
90266
91501
90604
92253
90004
90220
92203
90606
90036
91103
90603
90063
92582
90005
91042
91506
90713
92570
91304
91301
91755
92571
90241
91504
93552
92262
90605
92236
92563
90723
90640
90044
92583
90201
90011
90065
90037
90221
90042
90706
90272
90601
92591
90240
90404
90403
91325
9006

In [35]:
final_df.dropna(inplace=True)

In [36]:
final_df.to_csv('final_df_processed_3.csv')

In [70]:
zip_df.columns

Index(['property_id', 'sale_price', 'building_year_built', 'zip_code',
       'Address_Google', 'closest_zip_1', 'distance_1', 'closest_zip_2',
       'distance_2', 'closest_zip_3', 'distance_3', 'Month', 'Year', 'Returns',
       'California AGI', 'City_Name', 'Unemployment Rate', 'Labor Force',
       'Date', 'property_type_SINGLE FAMILY DWELLING',
       'property_type_SINGLE FAMILY RESIDENCE', 'Zip_month',
       'zip_1_Unemployment Rate_0', 'zip_1_Unemployment Rate_1',
       'zip_1_Unemployment Rate_2', 'zip_1_Unemployment Rate_3',
       'zip_1_sale_price_0', 'zip_1_sale_price_1', 'zip_1_sale_price_2',
       'zip_1_sale_price_3', 'zip_1_California AGI_0',
       'zip_1_California AGI_1', 'zip_1_California AGI_2',
       'zip_1_California AGI_3', 'zip_1_Labor Force_0', 'zip_1_Labor Force_1',
       'zip_1_Labor Force_2', 'zip_1_Labor Force_3',
       'zip_2_Unemployment Rate_0', 'zip_2_Unemployment Rate_1',
       'zip_2_Unemployment Rate_2', 'zip_2_Unemployment Rate_3',
       